<a href="https://colab.research.google.com/github/Amimo-cell/AMR-Biosensor-using-Wastewater/blob/main/AMR_Biosensor_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the required Libraries
!pip install numpy pandas plotly scipy scikit-learn -q
!pip install ipywidgets -q  # For interactive widgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00


In [5]:
# Import All Libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [8]:
#Define AMR Gene database
class AMRGeneDatabase:
    """Database of WHO-priority AMR genes for wastewater surveillance"""

    def __init__(self):
        self.genes = {
            'blaCTX-M-15': {
                'full_name': 'CTX-M-15 beta-lactamase',
                'sequence': 'ATGAAGTTAAGTACTGTGCAGTGCCGTTGGCGGCTGCAGTTGTTG...',  # Truncated
                'who_priority': 'High',
                'drug_resistance': ['Ceftriaxone', 'Cefotaxime', 'Aztreonam'],
                'typical_concentration': {'hospital': 5000, 'wwtp': 500, 'river': 50},  # copies/mL
                'prevalence': 0.45,  # 45% of samples positive
                'detection_threshold': 100  # copies/mL
            },
            'blaNDM-1': {
                'full_name': 'New Delhi metallo-beta-lactamase 1',
                'sequence': 'ATGAATTCATACCGCCTGTGTGCCACCTCGACGCCGCA...',
                'who_priority': 'Critical',
                'drug_resistance': ['Carbapenems', 'Penicillins', 'Cephalosporins'],
                'typical_concentration': {'hospital': 2000, 'wwtp': 200, 'river': 20},
                'prevalence': 0.15,
                'detection_threshold': 50
            },
            'mecA': {
                'full_name': 'Methicillin resistance protein',
                'sequence': 'ATGAAGATATACCAAGTGTTGAAAAAGGTGATGAAGTT...',
                'who_priority': 'High',
                'drug_resistance': ['Methicillin', 'Oxacillin', 'Cefoxitin'],
                'typical_concentration': {'hospital': 3000, 'wwtp': 300, 'river': 30},
                'prevalence': 0.25,
                'detection_threshold': 200
            },
            'vanA': {
                'full_name': 'Vancomycin resistance operon',
                'sequence': 'ATGGCAAGCGATAAAATATTGATTGCTATTC...',
                'who_priority': 'High',
                'drug_resistance': ['Vancomycin', 'Teicoplanin'],
                'typical_concentration': {'hospital': 1000, 'wwtp': 100, 'river': 10},
                'prevalence': 0.08,
                'detection_threshold': 150
            },
            'mcr-1': {
                'full_name': 'Mobilized colistin resistance 1',
                'sequence': 'ATGAAAAATTATATTTATTTTTGTGATATTC...',
                'who_priority': 'Medium',
                'drug_resistance': ['Colistin', 'Polymyxin B'],
                'typical_concentration': {'hospital': 800, 'wwtp': 80, 'river': 8},
                'prevalence': 0.05,
                'detection_threshold': 100
            }
        }

    def get_gene_info(self, gene_name):
        """Get detailed information about a specific AMR gene"""
        return self.genes.get(gene_name, {})

    def list_genes(self):
        """List all available AMR genes"""
        return list(self.genes.keys())

# Initialize database
amr_db = AMRGeneDatabase()
print(f" AMR Database loaded with {len(amr_db.genes)} priority genes:")
for gene in amr_db.list_genes():
    print(f"   - {gene}: {amr_db.get_gene_info(gene)['full_name']}")



 AMR Database loaded with 5 priority genes:
   - blaCTX-M-15: CTX-M-15 beta-lactamase
   - blaNDM-1: New Delhi metallo-beta-lactamase 1
   - mecA: Methicillin resistance protein
   - vanA: Vancomycin resistance operon
   - mcr-1: Mobilized colistin resistance 1


In [9]:
# CRISPR-Based Biosensor Simulation
class CRISPRBiosensor:
    """
    Simulates a CRISPR-Cas12a electrochemical biosensor for AMR gene detection.
    This is a COMPUTATIONAL SIMULATION only.
    """

    def __init__(self, sensitivity=2.0, noise_level=0.1):
        """
        Initialize biosensor with performance parameters.

        Args:
            sensitivity: Signal amplification factor (higher = more sensitive)
            noise_level: Measurement noise standard deviation
        """
        self.sensitivity = sensitivity
        self.noise_level = noise_level
        self.calibration_factor = 1.0
        self.cross_reactivity = self._create_cross_reactivity_matrix()

        print(f"🧬 Biosensor initialized:")
        print(f"   • Sensitivity: {sensitivity}")
        print(f"   • Noise level: {noise_level}")
        print(f"   • Detection limit: {self.calculate_detection_limit():.0f} copies/mL")

    def simulate_measurement(self, gene_concentrations, location_type="hospital"):
        """
        Simulate a measurement of AMR genes in a wastewater sample.

        Args:
            gene_concentrations: Dict of {gene_name: concentration in copies/mL}
            location_type: "hospital", "wwtp", or "river"

        Returns:
            Dict with detection results
        """
        results = {}

        for gene_name, true_concentration in gene_concentrations.items():
            # Skip if gene not in database
            if gene_name not in amr_db.genes:
                continue

            # Get gene info
            gene_info = amr_db.get_gene_info(gene_name)

            # Calculate base signal (log-linear response)
            if true_concentration > 0:
                base_signal = self.sensitivity * np.log10(true_concentration)
            else:
                base_signal = -10  # Background noise level

            # Add cross-reactivity from other genes
            cross_signal = 0
            for other_gene, other_conc in gene_concentrations.items():
                if other_gene != gene_name and other_conc > 0:
                    cross = self.cross_reactivity[gene_name][other_gene]
                    cross_signal += cross * np.log10(other_conc)

            # Add measurement noise (normal distribution)
            noise = np.random.normal(0, self.noise_level)

            # Total measured signal
            measured_signal = base_signal + cross_signal + noise

            # Determine if detected (3σ above background)
            detection_threshold = 3 * self.noise_level
            detected = measured_signal > detection_threshold

            # Estimate concentration from signal
            if detected:
                estimated_conc = 10**(measured_signal / self.sensitivity)
            else:
                estimated_conc = 0

            # Calculate confidence score
            confidence = self._calculate_confidence(measured_signal, detection_threshold)

            # Store results
            results[gene_name] = {
                'true_concentration': true_concentration,
                'measured_signal': measured_signal,
                'detected': detected,
                'estimated_concentration': estimated_conc,
                'confidence': confidence,
                'detection_threshold': detection_threshold,
                'who_priority': gene_info['who_priority']
            }

        return results

    def calculate_detection_limit(self):
        """Calculate limit of detection (LOD) in copies/mL"""
        # LOD = concentration that gives signal = 3 * noise
        lod_concentration = 10**((3 * self.noise_level) / self.sensitivity)
        return lod_concentration

    def _calculate_confidence(self, signal, threshold):
        """Calculate detection confidence (0-100%)"""
        if signal <= threshold:
            return 0
        else:
            # Sigmoid function for confidence
            confidence = 100 / (1 + np.exp(-(signal - threshold) * 5))
            return min(100, confidence)

    def _create_cross_reactivity_matrix(self):
        """Create cross-reactivity matrix between AMR genes"""
        genes = amr_db.list_genes()
        cross_matrix = {}

        # Initialize with identity matrix
        for gene1 in genes:
            cross_matrix[gene1] = {}
            for gene2 in genes:
                if gene1 == gene2:
                    cross_matrix[gene1][gene2] = 1.0  # Perfect self-reactivity
                else:
                    cross_matrix[gene1][gene2] = 0.0  # No cross-reactivity by default

        # Add some realistic cross-reactivity
        # ESBL genes might cross-react slightly
        cross_matrix['blaCTX-M-15']['blaNDM-1'] = 0.05
        cross_matrix['blaNDM-1']['blaCTX-M-15'] = 0.05

        # Add random background cross-reactivity (0-2%)
        for gene1 in genes:
            for gene2 in genes:
                if gene1 != gene2:
                    cross_matrix[gene1][gene2] += np.random.uniform(0, 0.02)

        return cross_matrix

# Test the biosensor
print("\n Testing biosensor...")
biosensor = CRISPRBiosensor(sensitivity=2.0, noise_level=0.1)

# Create a test sample
test_sample = {
    'blaCTX-M-15': 5000,
    'blaNDM-1': 2000,
    'mecA': 0,  # Not present
    'vanA': 100,
    'mcr-1': 0
}

results = biosensor.simulate_measurement(test_sample, "hospital")

print("\n Test Results:")
for gene, result in results.items():
    status = " DETECTED" if result['detected'] else " NOT DETECTED"
    print(f"   {gene}: {status} | True: {result['true_concentration']} | Estimated: {result['estimated_concentration']:.0f} copies/mL")




 Testing biosensor...
🧬 Biosensor initialized:
   • Sensitivity: 2.0
   • Noise level: 0.1
   • Detection limit: 1 copies/mL

 Test Results:
   blaCTX-M-15:  DETECTED | True: 5000 | Estimated: 6580 copies/mL
   blaNDM-1:  DETECTED | True: 2000 | Estimated: 2865 copies/mL
   mecA:  NOT DETECTED | True: 0 | Estimated: 0 copies/mL
   vanA:  DETECTED | True: 100 | Estimated: 111 copies/mL
   mcr-1:  NOT DETECTED | True: 0 | Estimated: 0 copies/mL


In [25]:
# Generate Realistic Wastewater Samples
class WastewaterSampler:
    """Generates realistic wastewater samples with AMR genes"""

    def __init__(self):
        self.locations = {
            'hospital': {
                'description': 'Hospital wastewater effluent',
                'concentration_multiplier': 10.0,
                'variance': 1.2,
                'typical_flow': 100  # L/person/day
            },
            'wwtp': {
                'description': 'Wastewater treatment plant influent',
                'concentration_multiplier': 1.0,
                'variance': 0.8,
                'typical_flow': 200
            },
            'river': {
                'description': 'River downstream of discharge',
                'concentration_multiplier': 0.1,
                'variance': 1.5,
                'typical_flow': 10000
            }
        }

    def generate_sample(self, location='hospital', outbreak_factor=1.0, days_offset=0):
        """
        Generate a simulated wastewater sample.

        Args:
            location: 'hospital', 'wwtp', or 'river'
            outbreak_factor: Multiply concentrations by this during outbreaks
            days_offset: Days since outbreak start (affects concentrations)

        Returns:
            Dict of gene concentrations
        """
        if location not in self.locations:
            raise ValueError(f"Location must be one of {list(self.locations.keys())}")

        loc_info = self.locations[location]
        sample = {}

        for gene_name in amr_db.list_genes():
            gene_info = amr_db.get_gene_info(gene_name)

            # Base concentration for this location
            base_conc = gene_info['typical_concentration'][location]

            # Add temporal variation (sinusoidal + random)
            time_factor = 1 + 0.3 * np.sin(2 * np.pi * days_offset / 7)  # Weekly pattern
            random_factor = np.random.lognormal(mean=0, sigma=0.3)

            # Outbreak effect (grows over time)
            if outbreak_factor > 1.0:
                outbreak_effect = outbreak_factor * (1 + 0.1 * days_offset)
            else:
                outbreak_effect = 1.0

            # Calculate final concentration
            concentration = base_conc * time_factor * random_factor * outbreak_effect

            # Add measurement error
            concentration = max(0, concentration + np.random.normal(0, base_conc * 0.1))

            sample[gene_name] = concentration

        return sample

    def generate_time_series(self, location='hospital', days=30, outbreak_start_day=10, outbreak_strength=5.0):
        """
        Generate time series of wastewater samples.

        Returns:
            DataFrame with concentrations over time
        """
        dates = pd.date_range(start='2024-01-01', periods=days, freq='D')
        all_data = []

        for i, date in enumerate(dates):
            # Check if outbreak has started
            if i >= outbreak_start_day:
                outbreak_factor = outbreak_strength * min(1.0, (i - outbreak_start_day) / 5)
            else:
                outbreak_factor = 1.0

            # Generate sample for this day
            sample = self.generate_sample(location, outbreak_factor, i)

            # Add to data
            for gene, conc in sample.items():
                all_data.append({
                    'date': date,
                    'gene': gene,
                    'concentration': conc,
                    'location': location,
                    'outbreak': i >= outbreak_start_day
                })

        return pd.DataFrame(all_data)

# Test the sampler
print("\n Generating wastewater samples...")
sampler = WastewaterSampler()

# Generate a hospital sample
hospital_sample = sampler.generate_sample('hospital')
print(f"\n Hospital Sample Concentrations (copies/mL):")
for gene, conc in hospital_sample.items():
    print(f"   {gene}: {conc:.0f}")

# Generate time series
print("\n Generating 30-day time series...")
time_series = sampler.generate_time_series('hospital', days=30, outbreak_start_day=10, outbreak_strength=8.0)
print(f"   Generated {len(time_series)} data points")
print(f"   Outbreak starts on day 10")




 Generating wastewater samples...

 Hospital Sample Concentrations (copies/mL):
   blaCTX-M-15: 2727
   blaNDM-1: 1144
   mecA: 1661
   vanA: 775
   mcr-1: 634

 Generating 30-day time series...
   Generated 150 data points
   Outbreak starts on day 10


In [26]:
# Early Outbreak Detection System
class OutbreakDetector:
    """AI system to detect AMR outbreaks from wastewater data"""

    def __init__(self):
        self.baseline_data = None
        self.alert_threshold = 3.0  # 3 standard deviations above baseline

    def establish_baseline(self, historical_data):
        """
        Establish a normal baseline from historical data.

        Args:
            historical_data: DataFrame with columns ['date', 'gene', 'concentration']
        """
        self.baseline_data = {}

        for gene in historical_data['gene'].unique():
            gene_data = historical_data[historical_data['gene'] == gene]['concentration']
            # Ensure there's enough data to calculate a standard deviation
            if len(gene_data) > 1 and gene_data.std() > 0:
                self.baseline_data[gene] = {
                    'mean': gene_data.mean(),
                    'std': gene_data.std(),
                    'n_samples': len(gene_data)
                }
            else:
                # Handle cases where std is 0 or NaN (e.g., if all baseline values are the same or only one sample)
                # For now, setting a very small std to avoid division by zero, or a fixed reasonable value
                # A more robust solution might involve collecting more baseline data or different statistical methods.
                self.baseline_data[gene] = {
                    'mean': gene_data.mean() if not gene_data.empty else 0,
                    'std': 0.01,  # A small default std to allow calculations
                    'n_samples': len(gene_data)
                }

        print(" Baseline established for genes:")
        for gene, stats in self.baseline_data.items():
            print(f"   {gene}: Mean={stats['mean']:.0f}, STD={stats['std']:.2f}")

    def check_for_outbreak(self, current_sample):
        """
        Check if the current sample indicates an outbreak.

        Returns:
            Dict with outbreak alerts and severity
        """
        if self.baseline_data is None:
            raise ValueError("Must establish baseline first!")

        alerts = []
        outbreak_score = 0

        for gene, concentration in current_sample.items():
            if gene in self.baseline_data:
                baseline = self.baseline_data[gene]

                # Calculate Z-score (how many standard deviations from mean)
                # Ensure std is not zero to prevent division by zero errors
                if baseline['std'] > 0:
                    z_score = (concentration - baseline['mean']) / baseline['std']
                else:
                    z_score = 0 # If std is zero, no deviation can be measured

                # Check if significant increase
                if z_score > self.alert_threshold:
                    severity = min(10, z_score)  # Scale to 1-10
                    outbreak_score += severity

                    # Determine alert level
                    if z_score > 5:
                        alert_level = "🔴 CRITICAL"
                    elif z_score > 3:
                        alert_level = "🟠 HIGH"
                    else:
                        alert_level = "🟡 MEDIUM"

                    alerts.append({
                        'gene': gene,
                        'concentration': concentration,
                        'z_score': z_score,
                        'alert_level': alert_level,
                        'baseline_mean': baseline['mean']
                    })

        # Determine overall outbreak status
        if outbreak_score > 15:
            overall_status = "🔴 MAJOR OUTBREAK DETECTED"
        elif outbreak_score > 8:
            overall_status = "🟠 MINOR OUTBREAK DETECTED"
        elif outbreak_score > 3:
            overall_status = "🟡 ELEVATED LEVELS"
        else:
            overall_status = "✅ NORMAL"

        return {
            'overall_status': overall_status,
            'outbreak_score': outbreak_score,
            'alerts': alerts,
            'timestamp': datetime.now()
        }

    def simulate_outbreak_scenario(self, location='hospital', duration_days=30):
        """
        Simulate a complete outbreak scenario.

        Returns:
            Time series DataFrame with outbreak detection results
        """
        # Generate baseline data (first 14 days normal)
        print(f"\n Simulating outbreak scenario at {location}...")
        print("   Days 1-14: Normal baseline")
        print("   Day 15: Outbreak begins")
        print("   Day 20: Outbreak peaks")

        all_results = []
        baseline_data_accumulator = [] # Accumulate baseline data here

        for day in range(duration_days):
            # Simulate outbreak strength (starts day 15, peaks day 20)
            if day < 14:
                outbreak_factor = 1.0  # Normal
            elif day < 20:
                outbreak_factor = 1 + (day - 14) * 2  # Ramping up
            else:
                outbreak_factor = 12 - (day - 20) * 0.5  # Declining slowly

            # Generate sample for this day
            sample = sampler.generate_sample(location, outbreak_factor, day)

            # Convert to format for outbreak detector
            current_sample = {}
            for gene, conc in sample.items():
                if gene in ['blaCTX-M-15', 'blaNDM-1', 'mecA']:  # Monitor key genes
                    current_sample[gene] = conc

            # Accumulate baseline data for the first 14 days
            if day < 14:
                for gene, conc in current_sample.items():
                    baseline_data_accumulator.append({
                        'date': datetime(2024, 1, 1) + timedelta(days=day),
                        'gene': gene,
                        'concentration': conc
                    })

            if day == 14:
                # Establish baseline after 14 days of data collection
                historical_df = pd.DataFrame(baseline_data_accumulator)
                self.establish_baseline(historical_df)

            if day >= 14:
                outbreak_check = self.check_for_outbreak(current_sample)

                all_results.append({
                    'day': day,
                    'date': datetime(2024, 1, 1) + timedelta(days=day),
                    'outbreak_factor': outbreak_factor,
                    'overall_status': outbreak_check['overall_status'],
                    'outbreak_score': outbreak_check['outbreak_score'],
                    'alerts_count': len(outbreak_check['alerts'])
                })

        return pd.DataFrame(all_results)

# Run outbreak simulation
print("\n Starting outbreak detection simulation...")
detector = OutbreakDetector()
outbreak_results = detector.simulate_outbreak_scenario('hospital', duration_days=30)

# Visualize outbreak timeline
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=outbreak_results['date'],
    y=outbreak_results['outbreak_score'],
    mode='lines+markers',
    name='Outbreak Score',
    line=dict(width=3, color='#D32F2F'),
    fill='tozeroy',
    fillcolor='rgba(211, 47, 47, 0.1)'
))

# Add alert thresholds
fig.add_hline(y=3, line_dash="dash", line_color="orange",
              annotation_text="Alert Threshold", annotation_position="bottom right")
fig.add_hline(y=8, line_dash="dash", line_color="red",
              annotation_text="Outbreak Threshold", annotation_position="top right")

# Convert datetime to Unix timestamp (milliseconds since epoch) for numerical x-value
outbreak_start_date = datetime(2024, 1, 15)
x_value_for_vline = int(outbreak_start_date.timestamp() * 1000)

fig.add_vline(x=x_value_for_vline, line_dash="dot", line_color="red",
              annotation_text="Outbreak Starts", annotation_position="top")

fig.update_layout(
    title="AMR Outbreak Detection Timeline",
    xaxis_title="Date",
    yaxis_title="Outbreak Severity Score",
    hovermode="x unified",
    height=500
)

fig.show()

print("\nOutbreak Detection Summary:")
print(f"   Total days simulated: {len(outbreak_results)}")
print(f"   Days with alerts: {(outbreak_results['outbreak_score'] > 3).sum()}")
print(f"   Maximum outbreak score: {outbreak_results['outbreak_score'].max():.1f}")


 Starting outbreak detection simulation...

 Simulating outbreak scenario at hospital...
   Days 1-14: Normal baseline
   Day 15: Outbreak begins
   Day 20: Outbreak peaks
 Baseline established for genes:
   blaCTX-M-15: Mean=5151, STD=1579.87
   blaNDM-1: Mean=1859, STD=770.02
   mecA: Mean=3180, STD=965.61



Outbreak Detection Summary:
   Total days simulated: 16
   Days with alerts: 15
   Maximum outbreak score: 30.0


In [27]:
# Biosensor Performance Analysis
def analyze_biosensor_performance():
    """Analyze biosensor performance metrics"""

    print("\n" + "="*60)
    print(" BIOSENSOR PERFORMANCE ANALYSIS")
    print("="*60)

    # Test different concentrations
    test_concentrations = [10, 50, 100, 500, 1000, 5000, 10000]
    detection_rates = []

    for conc in test_concentrations:
        # Run 100 simulations at each concentration
        detections = 0
        for _ in range(100):
            sample = {'blaCTX-M-15': conc}
            results = biosensor.simulate_measurement(sample)
            if results['blaCTX-M-15']['detected']:
                detections += 1

        detection_rate = detections / 100
        detection_rates.append(detection_rate)

        print(f"   Concentration {conc:5d} copies/mL: {detection_rate*100:5.1f}% detection rate")

    # Calculate Limit of Detection (LOD)
    lod = biosensor.calculate_detection_limit()
    print(f"\n   🔬 Calculated Limit of Detection (LOD): {lod:.0f} copies/mL")

    # Calculate accuracy at LOD
    sample_at_lod = {'blaCTX-M-15': lod}
    results = biosensor.simulate_measurement(sample_at_lod)
    print(f"   Signal at LOD: {results['blaCTX-M-15']['measured_signal']:.2f}")
    print(f"   Detection threshold: {results['blaCTX-M-15']['detection_threshold']:.2f}")

    # Create performance plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=test_concentrations,
        y=detection_rates,
        mode='lines+markers',
        name='Detection Rate',
        line=dict(width=3, color='#2E86AB'),
        marker=dict(size=10)
    ))

    # Add LOD line
    fig.add_vline(x=lod, line_dash="dash", line_color="red",
                  annotation_text=f"LOD = {lod:.0f} copies/mL",
                  annotation_position="top right")

    # Add 95% detection line
    fig.add_hline(y=0.95, line_dash="dot", line_color="green",
                  annotation_text="95% Detection", annotation_position="top left")

    fig.update_layout(
        title="Biosensor Performance: Detection Rate vs Concentration",
        xaxis_title="Concentration (copies/mL)",
        yaxis_title="Detection Rate",
        xaxis_type="log",
        height=500
    )

    fig.update_xaxes(range=[np.log10(10), np.log10(10000)])

    fig.show()

    return fig

# Run performance analysis
performance_fig = analyze_biosensor_performance()




 BIOSENSOR PERFORMANCE ANALYSIS
   Concentration    10 copies/mL: 100.0% detection rate
   Concentration    50 copies/mL: 100.0% detection rate
   Concentration   100 copies/mL: 100.0% detection rate
   Concentration   500 copies/mL: 100.0% detection rate
   Concentration  1000 copies/mL: 100.0% detection rate
   Concentration  5000 copies/mL: 100.0% detection rate
   Concentration 10000 copies/mL: 100.0% detection rate

   🔬 Calculated Limit of Detection (LOD): 1 copies/mL
   Signal at LOD: 0.34
   Detection threshold: 0.30


In [28]:
# Save Results & Generate Report
def save_simulation_results(results, filename="biosensor_results"):
    """Save simulation results to files"""

    import json
    import csv
    from google.colab import files

    # Convert results to DataFrame
    df_data = []
    for gene, data in results.items():
        df_data.append({
            'gene': gene,
            'true_concentration': data['true_concentration'],
            'estimated_concentration': data['estimated_concentration'],
            'measured_signal': data['measured_signal'],
            'detected': data['detected'],
            'confidence': data['confidence'],
            'who_priority': data['who_priority']
        })

    df = pd.DataFrame(df_data)

    # Save to CSV
    csv_filename = f"{filename}.csv"
    df.to_csv(csv_filename, index=False)

    # Save to JSON
    json_filename = f"{filename}.json"
    with open(json_filename, 'w') as f:
        json.dump(results, f, indent=2, default=str)

    # Generate summary report
    report_filename = f"{filename}_report.txt"
    with open(report_filename, 'w') as f:
        f.write("="*60 + "\n")
        f.write("AMR BIOSENSOR SIMULATION REPORT\n")
        f.write("="*60 + "\n\n")
        f.write(f"Simulation Date: {datetime.now()}\n")
        f.write(f"Biosensor Sensitivity: {biosensor.sensitivity}\n")
        f.write(f"Biosensor Noise Level: {biosensor.noise_level}\n")
        f.write(f"Detection Limit: {biosensor.calculate_detection_limit():.0f} copies/mL\n\n")

        f.write("DETECTION RESULTS:\n")
        f.write("-"*40 + "\n")
        for gene, data in results.items():
            status = "DETECTED" if data['detected'] else "NOT DETECTED"
            f.write(f"{gene}:\n")
            f.write(f"  Status: {status}\n")
            f.write(f"  True Concentration: {data['true_concentration']:.0f} copies/mL\n")
            f.write(f"  Estimated: {data['estimated_concentration']:.0f} copies/mL\n")
            f.write(f"  Confidence: {data['confidence']:.0f}%\n")
            f.write(f"  WHO Priority: {data['who_priority']}\n\n")

    print(f"\n Results saved to:")
    print(f"   📄 {csv_filename}")
    print(f"   📄 {json_filename}")
    print(f"   📄 {report_filename}")

    # Download files
    print("\n  Downloading files...")
    files.download(csv_filename)
    files.download(json_filename)
    files.download(report_filename)

    return df

# Run a final simulation and save results
print("\n Saving final simulation results...")
final_sample = sampler.generate_sample('hospital', outbreak_factor=5.0)
final_results = biosensor.simulate_measurement(final_sample)
saved_df = save_simulation_results(final_results, "amr_biosensor_simulation")

# Display final results
print("\n" + "="*60)
print(" SIMULATION COMPLETE!")
print("="*60)
print("\n Final Detection Summary:")
print(saved_df[['gene', 'detected', 'true_concentration', 'estimated_concentration', 'confidence']].to_string(index=False))




 Saving final simulation results...

 Results saved to:
   📄 amr_biosensor_simulation.csv
   📄 amr_biosensor_simulation.json
   📄 amr_biosensor_simulation_report.txt



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


 SIMULATION COMPLETE!

 Final Detection Summary:
       gene  detected  true_concentration  estimated_concentration  confidence
blaCTX-M-15      True        30345.256053             51110.253725       100.0
   blaNDM-1      True        10435.581521             24135.511748       100.0
       mecA      True        16974.321556             20581.425694       100.0
       vanA      True         4826.321229              5881.234592       100.0
      mcr-1      True         5190.234232              5984.615342       100.0


In [30]:
# Run Complete Simulation (One Cell)
def run_complete_simulation():
    """Run the complete AMR biosensor simulation"""

    print(" Starting Complete AMR Biosensor Simulation")
    print("="*60)

    # 1. Initialize
    print("\n1.  Initializing systems...")
    amr_db = AMRGeneDatabase()
    biosensor = CRISPRBiosensor(sensitivity=2.0, noise_level=0.1)
    sampler = WastewaterSampler()
    detector = OutbreakDetector()

    # 2. Generate hospital outbreak scenario
    print("\n2. Simulating hospital wastewater monitoring...")
    time_series = sampler.generate_time_series('hospital', days=30, outbreak_start_day=10, outbreak_strength=8.0)

    # 3. Process latest sample
    print("\n3.  Processing latest wastewater sample...")
    latest_sample = sampler.generate_sample('hospital', outbreak_factor=3.0, days_offset=29)
    detection_results = biosensor.simulate_measurement(latest_sample)

    # 4. Check for outbreaks
    print("\n4.  Running outbreak detection...")
    # Establish baseline from first 14 days
    baseline_data = time_series[time_series['date'] < pd.Timestamp('2024-01-15')]
    detector.establish_baseline(baseline_data)

    # Check current sample
    current_sample_dict = {}
    for gene in ['blaCTX-M-15', 'blaNDM-1', 'mecA']:
        if gene in latest_sample:
            current_sample_dict[gene] = latest_sample[gene]

    outbreak_check = detector.check_for_outbreak(current_sample_dict)

    # 5. Display results
    print("\n5.  RESULTS SUMMARY:")
    print("-"*40)

    detected_genes = [g for g, r in detection_results.items() if r['detected']]
    print(f"   Genes detected: {len(detected_genes)}/{len(detection_results)}")
    print(f"   Outbreak status: {outbreak_check['overall_status']}")
    print(f"   Outbreak score: {outbreak_check['outbreak_score']:.1f}")

    if outbreak_check['alerts']:
        print("\n   ALERTS:")
        for alert in outbreak_check['alerts']:
            print(f"     - {alert['alert_level']} for {alert['gene']}: "
                  f"Current {alert['concentration']:.0f} vs Baseline {alert['baseline_mean']:.0f} "
                  f"(Z-score: {alert['z_score']:.2f})")
    else:
        print("\n   No specific gene alerts.")

    print("="*60)
    print(" Simulation complete!")
    print("="*60)

# Call the complete simulation function
run_complete_simulation()

 Starting Complete AMR Biosensor Simulation

1.  Initializing systems...
🧬 Biosensor initialized:
   • Sensitivity: 2.0
   • Noise level: 0.1
   • Detection limit: 1 copies/mL

2. Simulating hospital wastewater monitoring...

3.  Processing latest wastewater sample...

4.  Running outbreak detection...
 Baseline established for genes:
   blaCTX-M-15: Mean=10917, STD=11353.61
   blaNDM-1: Mean=4585, STD=5014.04
   mecA: Mean=6202, STD=6371.16
   vanA: Mean=2139, STD=2557.18
   mcr-1: Mean=1973, STD=2201.57

5.  RESULTS SUMMARY:
----------------------------------------
   Genes detected: 5/5
   Outbreak status: 🔴 MAJOR OUTBREAK DETECTED
   Outbreak score: 24.2

   ALERTS:
     - 🔴 CRITICAL for blaCTX-M-15: Current 68103 vs Baseline 10917 (Z-score: 5.04)
     - 🔴 CRITICAL for blaNDM-1: Current 50629 vs Baseline 4585 (Z-score: 9.18)
     - 🔴 CRITICAL for mecA: Current 78559 vs Baseline 6202 (Z-score: 11.36)
 Simulation complete!


In [32]:
#Real data Integration
# CURRENT: We're using simulated data
test_sample = {'blaCTX-M-15': 5000}  # Made up!

# NEXT: Connect to REAL databases
import requests
def get_real_wastewater_data():
    # Connect to actual wastewater surveillance databases
    response = requests.get("https://www.ncbi.nlm.nih.gov/api/...")
    # Get REAL AMR prevalence data from:
    # 1. NCBI Pathogen Detection
    # 2. ECDC AMR Surveillance
    # 3. Local wastewater treatment plants

In [33]:
#Multiplex detection Engine
# CURRENT: Detects genes one at a time
# NEXT: True multiplex (6+ genes simultaneously)
class MultiplexBiosensor:
    def detect_all(self, sample):
        # Different electrodes for different genes
        # Each with unique CRISPR guide RNAs
        # Simultaneous electrochemical readout

        return {
            'electrode_1': {'gene': 'blaCTX-M-15', 'signal': 2.4},
            'electrode_2': {'gene': 'blaNDM-1', 'signal': 1.8},
            'electrode_3': {'gene': 'mecA', 'signal': 0.9},
            # ... up to 8 electrodes
        }

In [34]:
# IOT Network Simulation
# CURRENT: Single sensor
# NEXT: City-wide network
class CityWideNetwork:
    def simulate(self):
        return {
            'nodes': [
                {'id': 'HOSP_01', 'type': 'hospital', 'lat': 40.7128, 'lon': -74.0060},
                {'id': 'WWTP_01', 'type': 'treatment_plant', 'lat': 40.7589, 'lon': -73.9851},
                {'id': 'SCH_01', 'type': 'school', 'lat': 40.7794, 'lon': -73.9632},
                # ... 50+ nodes
            ],
            'connections': [
                {'from': 'HOSP_01', 'to': 'WWTP_01', 'flow_rate': '500L/s'},
                # ... sewer network topology
            ]
        }

In [35]:
# Machine learning predictions
# CURRENT: Basic threshold alerts
# NEXT: Predictive AI models
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet

class PredictiveModel:
    def train(self, historical_data):
        # Predict next week's AMR levels
        # Identify outbreak patterns 7-14 days in advance
        # Correlate with hospital admission data
        pass

    def predict_outbreak(self):
        return {
            'predicted_outbreak_date': '2024-03-15',
            'confidence': 0.87,
            'affected_genes': ['blaCTX-M-15', 'blaNDM-1'],
            'estimated_cases': 42
        }

In [38]:
# 3D Simulations
# CURRENT: 2D plots
# NEXT: Interactive 3D maps
import pydeck as pdk

def create_3d_sewer_map():
    # Show sensors in 3D sewer pipes
    # Animate AMR spread through network
    # Real-time particle tracking
    pass

In [50]:
!pip install pydeck -q